In [1]:
!pip install arcgis

In [2]:
!pip install arcpy

  ERROR: Could not find a version that satisfies the requirement arcpy (from versions: none)
ERROR: No matching distribution found for arcpy


# Project Question
## How do we determine voter turnout?
We will practice data engineering techniques to explore, visualize, and analyze presidential election participation data, otherwise known as "voter turnout". 

# Background
Data comes from https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/VOQCHQ
This dataset contains county-level returns for presidential elections from 2000 to 2016. (2018-10-11)
Review mapping info here: https://www.gislounge.com/how-to-do-map-stuff-live-online-tutorials/

In [3]:
import requests

In [4]:
import pandas as pd
import arcgis 
import os
#import arcpy #Geocoding and spatial statistics for hotspots
# other versions available? 

## Step 1: Load and Clean Election Data
Goals:

* Handle missing values
* Correct truncated zeroes with FIPS field
* Restructure table format

In [5]:
# Make reference to the file path for the csv, which should be in the same directory as the notebook
    #Internet link: "https://doi.org/10.7910/DVN/VOQCHQ" 
table_csv_path = "/Users/mehrunisaqayyum/Downloads/dataverse_files/countypres_2000-2016.csv" 
    #Internet link: "https://doi.org/10.7910/DVN/VOQCHQ" 

# Use Pandas to read the csv into a dataframe
    #Try encoding="ISO-8859-1" or encoding="utf-8",
vote_df = pd.read_csv(table_csv_path, dtype={'year': str, 'FIPS': str})  # dtype parameter specifies that year and FIPS fields are string

In [6]:
# Use the head function to display the first five records of the dataframe
vote_df.head

<bound method NDFrame.head of        year    state state_po       county  FIPS     office        candidate  \
0      2000  Alabama       AL      Autauga  1001  President          Al Gore   
1      2000  Alabama       AL      Autauga  1001  President   George W. Bush   
2      2000  Alabama       AL      Autauga  1001  President      Ralph Nader   
3      2000  Alabama       AL      Autauga  1001  President            Other   
4      2000  Alabama       AL      Baldwin  1003  President          Al Gore   
...     ...      ...      ...          ...   ...        ...              ...   
50519  2016   Alaska       AK  District 40  2040  President     Donald Trump   
50520  2016   Alaska       AK  District 40  2040  President            Other   
50521  2016   Alaska      NaN  District 99  2099  President  Hillary Clinton   
50522  2016   Alaska      NaN  District 99  2099  President     Donald Trump   
50523  2016   Alaska      NaN  District 99  2099  President            Other   

         

In [7]:
vote_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50524 entries, 0 to 50523
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            50524 non-null  object 
 1   state           50524 non-null  object 
 2   state_po        50460 non-null  object 
 3   county          50524 non-null  object 
 4   FIPS            50476 non-null  object 
 5   office          50524 non-null  object 
 6   candidate       50524 non-null  object 
 7   party           34735 non-null  object 
 8   candidatevotes  50120 non-null  float64
 9   totalvotes      50524 non-null  int64  
 10  version         50524 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 4.2+ MB


### Handle Missing Data

In [8]:
vote_df.shape

(50524, 11)

In [9]:
vote_df

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Alabama,AL,Autauga,1001,President,Al Gore,democrat,4942.0,17208,20191203
1,2000,Alabama,AL,Autauga,1001,President,George W. Bush,republican,11993.0,17208,20191203
2,2000,Alabama,AL,Autauga,1001,President,Ralph Nader,green,160.0,17208,20191203
3,2000,Alabama,AL,Autauga,1001,President,Other,NaN,113.0,17208,20191203
4,2000,Alabama,AL,Baldwin,1003,President,Al Gore,democrat,13997.0,56480,20191203
...,...,...,...,...,...,...,...,...,...,...,...
50519,2016,Alaska,AK,District 40,2040,President,Donald Trump,republican,1377.0,4610,20191203
50520,2016,Alaska,AK,District 40,2040,President,Other,NaN,895.0,4610,20191203
50521,2016,Alaska,NaN,District 99,2099,President,Hillary Clinton,democrat,274.0,5056,20191203
50522,2016,Alaska,NaN,District 99,2099,President,Donald Trump,republican,40.0,5056,20191203


In [10]:
#Look for missing values
vote_df['FIPS'].isnull()

0        False
1        False
2        False
3        False
4        False
         ...  
50519    False
50520    False
50521    False
50522    False
50523    False
Name: FIPS, Length: 50524, dtype: bool

In [11]:
#Identify number of null records in column 'FIPS' default is 'True'
vote_df.loc[vote_df["FIPS"].isnull()]

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
12452,2000,Connecticut,NaN,Statewide writein,NaN,President,Al Gore,democrat,NaN,0,20191203
12453,2000,Maine,NaN,Maine UOCAVA,NaN,President,Al Gore,democrat,NaN,0,20191203
12454,2000,Rhode Island,NaN,Federal Precinct,NaN,President,Al Gore,democrat,NaN,0,20191203
12455,2000,Connecticut,NaN,Statewide writein,NaN,President,George W. Bush,republican,NaN,0,20191203
12456,2000,Maine,NaN,Maine UOCAVA,NaN,President,George W. Bush,republican,NaN,0,20191203
12457,2000,Rhode Island,NaN,Federal Precinct,NaN,President,George W. Bush,republican,NaN,0,20191203
12458,2000,Connecticut,NaN,Statewide writein,NaN,President,Ralph Nader,green,NaN,0,20191203
12459,2000,Maine,NaN,Maine UOCAVA,NaN,President,Ralph Nader,green,NaN,0,20191203
12460,2000,Rhode Island,NaN,Federal Precinct,NaN,President,Ralph Nader,green,NaN,0,20191203
12461,2000,Connecticut,NaN,Statewide writein,NaN,President,Other,NaN,NaN,0,20191203


In [12]:
#Identify number of null records in column 'FIPS' default is 'True'
vote_df.loc[vote_df["FIPS"].isnull()].shape

(48, 11)

In [13]:
#Identify number of null records in column 'votes_dem', which becomes candidatevotes default is 'True'
vote_df.loc[vote_df["candidatevotes"].isnull()]

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
1442,2000,Georgia,GA,Baker,13007,President,Ralph Nader,green,NaN,1519,20191203
1462,2000,Georgia,GA,Ben Hill,13017,President,Ralph Nader,green,NaN,4661,20191203
1466,2000,Georgia,GA,Berrien,13019,President,Ralph Nader,green,NaN,4410,20191203
1486,2000,Georgia,GA,Bryan,13029,President,Ralph Nader,green,NaN,7059,20191203
1502,2000,Georgia,GA,Calhoun,13037,President,Ralph Nader,green,NaN,1887,20191203
...,...,...,...,...,...,...,...,...,...,...,...
50276,2008,Alaska,NaN,District 99,2099,President,John McCain,republican,NaN,0,20191203
50277,2008,Alaska,NaN,District 99,2099,President,Other,NaN,NaN,0,20191203
50398,2012,Alaska,NaN,District 99,2099,President,Barack Obama,democrat,NaN,74045,20191203
50399,2012,Alaska,NaN,District 99,2099,President,Mitt Romney,republican,NaN,74045,20191203


In [14]:
# Set the field to check nulls for
field_to_check = "FIPS"

# Determine how many rows are in the table
rowcount = vote_df.shape[0]

# Determine how many rows have null FIPS 
null_fips_rowcount = vote_df.loc[vote_df[field_to_check].isnull()].shape[0]

# Calculate how much of the data this represents as a percentage
percentage_null_fips = round((null_fips_rowcount / rowcount) * 100, 2)

# Use a print statement to report this information
print("There were "+str(null_fips_rowcount)+" records with null "+str(field_to_check)+" values in the data.\nThis amounts to " +str(percentage_null_fips)+"% of the available data.")

There were 48 records with null FIPS values in the data.
This amounts to 0.1% of the available data.


In [15]:
# Use the notnull function and the loc function to create a new dataframe without null FIPS records
vote_df = vote_df.loc[vote_df['FIPS'].notnull()]

In [16]:
vote_df = vote_df.loc[vote_df['candidatevotes'].notnull()]

In [17]:
# Get the first five records of the table
vote_df.head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Alabama,AL,Autauga,1001,President,Al Gore,democrat,4942.0,17208,20191203
1,2000,Alabama,AL,Autauga,1001,President,George W. Bush,republican,11993.0,17208,20191203
2,2000,Alabama,AL,Autauga,1001,President,Ralph Nader,green,160.0,17208,20191203
3,2000,Alabama,AL,Autauga,1001,President,Other,NaN,113.0,17208,20191203
4,2000,Alabama,AL,Baldwin,1003,President,Al Gore,democrat,13997.0,56480,20191203


In [18]:
# Check how many records have a FIPS value with four characters
truncate_df = vote_df.loc[vote_df['FIPS'].str.len() == 4]
truncate_data_per = (truncate_df.shape[0] / vote_df.shape[0])*100

# Use another print statement (using the f format key) to report this information
print(f"{round(truncate_data_per, 2)}% of data ({truncate_df.shape[0]} rows) has truncated FIPS values.")

10.44% of data (5231 rows) has truncated FIPS values.


#### Note: Next, we need to fix the FIPS field since the data has leading zeroes truncated by its interpretation as a numeric field. We can create a simple function in python to determine if the value is four characters, and append a leading zero if that's the case.

In [19]:
# Define a helper function to fix truncated zeros, with one parameter: the value to be processed
def fix_trunc_zeros(val):
    # Use an if statement to check if there are four characters in the string representation of the value
    if len(str(val)) == 4:
        # If this is the case, return the value with an appended "0" in the front
        return "0"+str(val)
    # Otherwise...
    else:
        # Return the value itself
        return str(val)

In [20]:
# Test helper function with truncated value
fix_trunc_zeros(7042)  # You should see an appended zero: "07042"

'07042'

In [21]:
# Run helper function on the FIPS field using the apply and lambda method 
vote_df['FIPS'] = vote_df['FIPS'].apply(lambda x: fix_trunc_zeros(x))

# Print information on the operation performed, and show the first few records to confirm it worked
print(f"{round(truncate_data_per, 2)}% of data ({truncate_df.shape[0]} rows) had truncated FIPS IDs corrected.")
vote_df.head()

10.44% of data (5231 rows) had truncated FIPS IDs corrected.


,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Alabama,AL,Autauga,01001,President,Al Gore,democrat,4942.0,17208,20191203
1,2000,Alabama,AL,Autauga,01001,President,George W. Bush,republican,11993.0,17208,20191203
2,2000,Alabama,AL,Autauga,01001,President,Ralph Nader,green,160.0,17208,20191203
3,2000,Alabama,AL,Autauga,01001,President,Other,NaN,113.0,17208,20191203
4,2000,Alabama,AL,Baldwin,01003,President,Al Gore,democrat,13997.0,56480,20191203


### Reformat the Structure

We now need to reformat the structure of the table. Currently, each record corresponds to a candidate and their votes in a particular county. We need each record to correspond to each county, with fields showing the votes for different candidates for that election year. (Remap: https://stackoverflow.com/questions/48385525/what-does-df-columns-df-columns-map0001-format-mean-exactly)

The animation above displays the reformatting necessary. It is possible to do this using Excel pivot tables, but a Python script to perform this might make things a bit easier to automate and share. 

In [22]:
vote_df.columns

Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')

In [23]:
#* Reformat the dataframe by setting a multiindex (set_index with multiple fields) and pivoting the table (unstack)
df_out = vote_df.set_index(['FIPS', 
                            'year', 
                            'county', 
                            'state', 
                            'state_po', 
                            'office', 
                            vote_df.groupby(['FIPS', 'year']).cumcount()+1]).unstack()
df_out.columns

MultiIndex([(     'candidate', 1),
            (     'candidate', 2),
            (     'candidate', 3),
            (     'candidate', 4),
            (         'party', 1),
            (         'party', 2),
            (         'party', 3),
            (         'party', 4),
            ('candidatevotes', 1),
            ('candidatevotes', 2),
            ('candidatevotes', 3),
            ('candidatevotes', 4),
            (    'totalvotes', 1),
            (    'totalvotes', 2),
            (    'totalvotes', 3),
            (    'totalvotes', 4),
            (       'version', 1),
            (       'version', 2),
            (       'version', 3),
            (       'version', 4)],
           )

In [24]:
# Use the indexes for the columns to set column names (Ex: candidate_1, candidate_2, votes_1, votes_2, etc.)
    #df_out.columns = df_out.columns.map('{0[0]}_{0[1]}'.format)
df_out.columns = df_out.columns.map('{0[0]}_{0[1]}'.format)

# Rename columns 
df_out = df_out.rename(columns={"candidate_1": "candidate_dem",
                                "candidatevotes_1": "votes_dem",
                                "candidate_2": "candidate_gop",
                                "candidatevotes_2": "votes_gop",
                                "totalvotes_1": "votes_total",
                                "state_po": "state_abbrev"
                                })
df_out

candidate_dem  \
FIPS  year county  state   state_po office                       
01001 2000 Autauga Alabama AL       President          Al Gore   
      2004 Autauga Alabama AL       President       John Kerry   
      2008 Autauga Alabama AL       President     Barack Obama   
      2012 Autauga Alabama AL       President     Barack Obama   
      2016 Autauga Alabama AL       President  Hillary Clinton   
...                                                        ...   
56045 2000 Weston  Wyoming WY       President          Al Gore   
      2004 Weston  Wyoming WY       President       John Kerry   
      2008 Weston  Wyoming WY       President     Barack Obama   
      2012 Weston  Wyoming WY       President     Barack Obama   
      2016 Weston  Wyoming WY       President  Hillary Clinton   

                                                candidate_gop  candidate_3  \
FIPS  year county  state   state_po office                                   
01001 2000 Autauga Alabama AL       President  George W. Bush  Ralph Nader   
      2004 Autauga Alabama AL       President  George W. Bush        Other   
      2008 Autauga Alabama AL       President     John McCain        Other   
      2012 Autauga Alabama AL       President     Mitt Romney        Other   
      2016 Autauga Alabama AL       President    Donald Trump        Other   
...                                                       ...          ...   
56045 2000 Weston  Wyoming WY       President  George W. Bush  Ralph Nader   
      2004 Weston  Wyoming WY       President  George W. Bush        Other   
      2008 Weston  Wyoming WY       President     John McCain        Other   
      2012 Weston  Wyoming WY       President     Mitt Romney        Other   
      2016 Weston  Wyoming WY       President    Donald Trump        Other   

                                              candidate_4   party_1  \
FIPS  year county  state   state_po office                            
01001 2000 Autauga Alabama AL       President       Other  democrat   
      2004 Autauga Alabama AL       President         NaN  democrat   
      2008 Autauga Alabama AL       President         NaN  democrat   
      2012 Autauga Alabama AL       President         NaN  democrat   
      2016 Autauga Alabama AL       President         NaN  democrat   
...                                                   ...       ...   
56045 2000 Weston  Wyoming WY       President       Other  democrat   
      2004 Weston  Wyoming WY       President         NaN  democrat   
      2008 Weston  Wyoming WY       President         NaN  democrat   
      2012 Weston  Wyoming WY       President         NaN  democrat   
      2016 Weston  Wyoming WY       President         NaN  democrat   

                                                  party_2 party_3 party_4  \
FIPS  year county  state   state_po office                                  
01001 2000 Autauga Alabama AL       President  republican   green     NaN   
      2004 Autauga Alabama AL       President  republican     NaN     NaN   
      2008 Autauga Alabama AL       President  republican     NaN     NaN   
      2012 Autauga Alabama AL       President  republican     NaN     NaN   
      2016 Autauga Alabama AL       President  republican     NaN     NaN   
...                                                   ...     ...     ...   
56045 2000 Weston  Wyoming WY       President  republican   green     NaN   
      2004 Weston  Wyoming WY       President  republican     NaN     NaN   
      2008 Weston  Wyoming WY       President  republican     NaN     NaN   
      2012 Weston  Wyoming WY       President  republican     NaN     NaN   
      2016 Weston  Wyoming WY       President  republican     NaN     NaN   

                                               votes_dem  votes_gop  \
FIPS  year county  state   state_po office                            
01001 2000 Autauga Alabama AL       President     4942.0    11993.0   
      2004 Autauga Alabama AL       

In [25]:
# Keep only the necessary columns
df_out = df_out[["candidate_dem", "votes_dem",
                 "candidate_gop", "votes_gop",
                 "votes_total"]]
df_out.columns

Index(['candidate_dem', 'votes_dem', 'candidate_gop', 'votes_gop',
       'votes_total'],
      dtype='object')

In [26]:
# Remove the multiindex since we no longer need these fields to be "locked" for the pivot
df_out.reset_index(inplace=True)
df_out

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total
0,01001,2000,Autauga,Alabama,AL,President,Al Gore,4942.0,George W. Bush,11993.0,17208.0
1,01001,2004,Autauga,Alabama,AL,President,John Kerry,4758.0,George W. Bush,15196.0,20081.0
2,01001,2008,Autauga,Alabama,AL,President,Barack Obama,6093.0,John McCain,17403.0,23641.0
3,01001,2012,Autauga,Alabama,AL,President,Barack Obama,6363.0,Mitt Romney,17379.0,23932.0
4,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973.0
...,...,...,...,...,...,...,...,...,...,...,...
15763,56045,2000,Weston,Wyoming,WY,President,Al Gore,449.0,George W. Bush,2521.0,3060.0
15764,56045,2004,Weston,Wyoming,WY,President,John Kerry,580.0,George W. Bush,2739.0,3392.0
15765,56045,2008,Weston,Wyoming,WY,President,Barack Obama,658.0,John McCain,2618.0,3414.0
15766,56045,2012,Weston,Wyoming,WY,President,Barack Obama,422.0,Mitt Romney,2821.0,3359.0


In [27]:
df_out.columns

Index(['FIPS', 'year', 'county', 'state', 'state_po', 'office',
       'candidate_dem', 'votes_dem', 'candidate_gop', 'votes_gop',
       'votes_total'],
      dtype='object')

In [28]:
# Print out the first few records to confirm everything worked
df_out.head()

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total
0,01001,2000,Autauga,Alabama,AL,President,Al Gore,4942.0,George W. Bush,11993.0,17208.0
1,01001,2004,Autauga,Alabama,AL,President,John Kerry,4758.0,George W. Bush,15196.0,20081.0
2,01001,2008,Autauga,Alabama,AL,President,Barack Obama,6093.0,John McCain,17403.0,23641.0
3,01001,2012,Autauga,Alabama,AL,President,Barack Obama,6363.0,Mitt Romney,17379.0,23932.0
4,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973.0


###  Reset Index
Essentially, we need to:

* Set a few fields aside, "locking" them from the table pivot.
* Pivot the table using the remaining fields.
* Give the pivoted fields designations for each party.
* Bring the locked fields back to our table.

For context, Pandas has the following powerful capabilities that help us perform this operation:

The ability to set an index using multiple fields, which acts as our "locking" mechanism for step 1.
The ability to perform an operation using a "groupby" function, which lets us group each observation by candidate/party
The ability to unstack a table, which lets us handle the table pivot.
1. Setting an index using multiple fields allows us to designate specific fields as index fields, which will not be impacted when a table pivot occurs. In other words, the fields specified in the set_index part of code above are "locked" when the later part of the code performs the table pivot via the "unstack" function.

2. The built-in groupby function allows us to perform an operation using the unique values from a specified set of fields. This is useful because we can then count how many rows exist for a given FIPS and Year combination, which essentially lets us group data by the candidate that it pertains to.

3. The ability to unstack a table allows us to perform the table pivot, which essentially "rotates" the table and makes rows into columns (or columns into rows).




# In [ ]:
vote_df.set_index(['FIPS', 
                   'year', 
                   'county', 
                   'state', 
                   'state_po', 
                   'office', 
                   vote_df.groupby(['FIPS', 'year']).cumcount()+1]).unstack()

4. Finally, we put it all together and rename the output columns, using the pandas ability to rename fields and removing extraneous fields we no longer need. This produces the table format we need.

In [ ]:

# Rename columns 
df_out = df_out.rename(columns={"candidate_dem_1": "candidate_dem",
                                "votes_dem_1": "votes_dem",
                                "candidate_gop_1": "candidate_gop",
                                "votes_gop_1": "votes_gop",
                                "votes_total_1": "votes_total",
                                "state_po": "state_abbrev"
                                })



##  Analysis: Calculate Additional Columns/Features
#### Calculate total votes for non major party

In [29]:
# Calculate votes that did not go for the Democrat or Republican party
df_out['votes_other'] = df_out['votes_total'] - (df_out['votes_dem'] + df_out['votes_gop'])
df_out.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other
0,01001,2000,Autauga,Alabama,AL,President,Al Gore,4942.0,George W. Bush,11993.0,17208.0,273.0
1,01001,2004,Autauga,Alabama,AL,President,John Kerry,4758.0,George W. Bush,15196.0,20081.0,127.0
2,01001,2008,Autauga,Alabama,AL,President,Barack Obama,6093.0,John McCain,17403.0,23641.0,145.0
3,01001,2012,Autauga,Alabama,AL,President,Barack Obama,6363.0,Mitt Romney,17379.0,23932.0,190.0
4,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973.0,865.0


### Create additional attributes (voter percentages and raw differences)

In [30]:
# Calculate voter share attributes
df_out['voter_share_major_party'] = (df_out['votes_dem'] + df_out['votes_gop']) / df_out['votes_total']
df_out['voter_share_dem'] = df_out['votes_dem'] / df_out['votes_total']
df_out['voter_share_gop'] = df_out['votes_gop'] / df_out['votes_total']
df_out['voter_share_other'] = df_out['votes_other'] / df_out['votes_total']
df_out

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other,voter_share_major_party,voter_share_dem,voter_share_gop,voter_share_other
0,01001,2000,Autauga,Alabama,AL,President,Al Gore,4942.0,George W. Bush,11993.0,17208.0,273.0,0.984135,0.287192,0.696943,0.015865
1,01001,2004,Autauga,Alabama,AL,President,John Kerry,4758.0,George W. Bush,15196.0,20081.0,127.0,0.993676,0.236940,0.756735,0.006324
2,01001,2008,Autauga,Alabama,AL,President,Barack Obama,6093.0,John McCain,17403.0,23641.0,145.0,0.993867,0.257730,0.736136,0.006133
3,01001,2012,Autauga,Alabama,AL,President,Barack Obama,6363.0,Mitt Romney,17379.0,23932.0,190.0,0.992061,0.265878,0.726183,0.007939
4,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973.0,865.0,0.965363,0.237697,0.727666,0.034637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15763,56045,2000,Weston,Wyoming,WY,President,Al Gore,449.0,George W. Bush,2521.0,3060.0,90.0,0.970588,0.146732,0.823856,0.029412
15764,56045,2004,Weston,Wyoming,WY,President,John Kerry,580.0,George W. Bush,2739.0,3392.0,73.0,0.978479,0.170991,0.807488,0.021521
15765,56045,2008,Weston,Wyoming,WY,President,Barack Obama,658.0,John McCain,2618.0,3414.0,138.0,0.959578,0.192736,0.766842,0.040422
15766,56045,2012,Weston,Wyoming,WY,President,Barack Obama,422.0,Mitt Romney,2821.0,3359.0,116.0,0.965466,0.125633,0.839833,0.034534


In [31]:
# Calculate raw difference attributes
df_out['rawdiff_dem_vs_gop'] = df_out['votes_dem'] - df_out['votes_gop']
df_out['rawdiff_gop_vs_dem'] = df_out['votes_gop'] - df_out['votes_dem']
df_out['rawdiff_dem_vs_other'] = df_out['votes_dem'] - df_out['votes_other']
df_out['rawdiff_gop_vs_other'] = df_out['votes_gop'] - df_out['votes_other']
df_out['rawdiff_other_vs_dem'] = df_out['votes_other'] - df_out['votes_dem']
df_out['rawdiff_other_vs_gop'] = df_out['votes_other'] - df_out['votes_gop']
df_out

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,...,voter_share_major_party,voter_share_dem,voter_share_gop,voter_share_other,rawdiff_dem_vs_gop,rawdiff_gop_vs_dem,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop
0,01001,2000,Autauga,Alabama,AL,President,Al Gore,4942.0,George W. Bush,11993.0,...,0.984135,0.287192,0.696943,0.015865,-7051.0,7051.0,4669.0,11720.0,-4669.0,-11720.0
1,01001,2004,Autauga,Alabama,AL,President,John Kerry,4758.0,George W. Bush,15196.0,...,0.993676,0.236940,0.756735,0.006324,-10438.0,10438.0,4631.0,15069.0,-4631.0,-15069.0
2,01001,2008,Autauga,Alabama,AL,President,Barack Obama,6093.0,John McCain,17403.0,...,0.993867,0.257730,0.736136,0.006133,-11310.0,11310.0,5948.0,17258.0,-5948.0,-17258.0
3,01001,2012,Autauga,Alabama,AL,President,Barack Obama,6363.0,Mitt Romney,17379.0,...,0.992061,0.265878,0.726183,0.007939,-11016.0,11016.0,6173.0,17189.0,-6173.0,-17189.0
4,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,...,0.965363,0.237697,0.727666,0.034637,-12236.0,12236.0,5071.0,17307.0,-5071.0,-17307.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15763,56045,2000,Weston,Wyoming,WY,President,Al Gore,449.0,George W. Bush,2521.0,...,0.970588,0.146732,0.823856,0.029412,-2072.0,2072.0,359.0,2431.0,-359.0,-2431.0
15764,56045,2004,Weston,Wyoming,WY,President,John Kerry,580.0,George W. Bush,2739.0,...,0.978479,0.170991,0.807488,0.021521,-2159.0,2159.0,507.0,2666.0,-507.0,-2666.0
15765,56045,2008,Weston,Wyoming,WY,President,Barack Obama,658.0,John McCain,2618.0,...,0.959578,0.192736,0.766842,0.040422,-1960.0,1960.0,520.0,2480.0,-520.0,-2480.0
15766,56045,2012,Weston,Wyoming,WY,President,Barack Obama,422.0,Mitt Romney,2821.0,...,0.965466,0.125633,0.839833,0.034534,-2399.0,2399.0,306.0,2705.0,-306.0,-2705.0


In [32]:
# Calculate percent difference attributes
df_out['pctdiff_dem_vs_gop'] = (df_out['votes_dem'] - df_out['votes_gop']) / df_out['votes_total']
df_out['pctdiff_gop_vs_dem'] = (df_out['votes_gop'] - df_out['votes_dem']) / df_out['votes_total']
df_out['pctdiff_dem_vs_other'] = (df_out['votes_dem'] - df_out['votes_other']) / df_out['votes_total']
df_out['pctdiff_gop_vs_other'] = (df_out['votes_gop'] - df_out['votes_other']) / df_out['votes_total']
df_out['pctdiff_other_vs_dem'] = (df_out['votes_other'] - df_out['votes_dem']) / df_out['votes_total']
df_out['pctdiff_other_vs_gop'] = (df_out['votes_other'] - df_out['votes_gop']) / df_out['votes_total']

df_out.head()

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,...,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop
0,01001,2000,Autauga,Alabama,AL,President,Al Gore,4942.0,George W. Bush,11993.0,...,4669.0,11720.0,-4669.0,-11720.0,-0.409751,0.409751,0.271327,0.681079,-0.271327,-0.681079
1,01001,2004,Autauga,Alabama,AL,President,John Kerry,4758.0,George W. Bush,15196.0,...,4631.0,15069.0,-4631.0,-15069.0,-0.519795,0.519795,0.230616,0.750411,-0.230616,-0.750411
2,01001,2008,Autauga,Alabama,AL,President,Barack Obama,6093.0,John McCain,17403.0,...,5948.0,17258.0,-5948.0,-17258.0,-0.478406,0.478406,0.251597,0.730003,-0.251597,-0.730003
3,01001,2012,Autauga,Alabama,AL,President,Barack Obama,6363.0,Mitt Romney,17379.0,...,6173.0,17189.0,-6173.0,-17189.0,-0.460304,0.460304,0.257939,0.718243,-0.257939,-0.718243
4,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,...,5071.0,17307.0,-5071.0,-17307.0,-0.489969,0.489969,0.203059,0.693028,-0.203059,-0.693028


## Step 2: Geoenable election data
### Goals:

Retrieve 2016 USA county population GIS data
Perform join, bringing geometry to election data
Calculate Voter Turnout per county
Convert dataframe to feature class
We will now bring geometry data for each county into the table.

Retrieve 2016 USA county population GIS data
A useful source of data is the ArcGIS Living Atlas of the World, where we can find a service containing Voting Age Population totals (citizens aged 18+) for each county.

This dataset is included with this project to allow you to append the election data to county geometry and allow us to calculate voting turnout for each county. The next few cells will load this feature class into a spatially-enabled dataframe and allow us to geoenable the election data we have been preparing. Additionally, this feature class allows us to actually calculate voter turnout since it includes voting-age population counts for 2016.

We will now reference this ArcGIS Pro project and its file geodatabase using ArcPy as well. 

In [ ]:
gis = arcgis.gis.GIS()


In [ ]:
#In [ ]:
# Search for USA_Counties
search = gis.content.search("USA Counties", item_type="feature_service", outside_org=True, sort_field="numViews")


In [ ]:
# Use the correct index to reference the search result
counties_item = search[2]
counties_item

#In [ ]:
# Read the layer into a dataframe
counties_df = pd.DataFrame.spatial.from_layer(counties_item.layers[0])
counties_df = counties_df[['FIPS', 'NAME', 'STATE_FIPS', 'STATE_NAME', 'OBJECTID', 'POPULATION', 'POP_SQMI', 'SHAPE', 'Shape_Area', 'Shape_Leng', 'Shape__Area', 'Shape__Length']]
counties_df.head()

### Perform join, bringing geometry and population column to election data
We now have a dataframe with election data ('df_out') and a spatially-enabled dataframe of county voting-age population data ('counties_df'). Let's merge the datasets.



In [ ]:
# Join the data to our election data table
geo_df = pd.merge(df_out, counties_df, left_on='FIPS', right_on="FIPS", how='left')
# Visualize the merged data, notice the SHAPE column at the end
geo_df.head()

In [ ]:
# Create a copy of the data, and perform a query
data_2016_df = geo_df.copy()
data_2016_df.query("year == '2016'", inplace=True)
data_2016_df.head()

### Convert dataframe to feature class
We can now finally convert our data to feature classes! The ArcGIS API for Python (which was invoked by using "import arcgis") lets us export the spatially-enabled dataframe to a feature class so we can do further analysis.

Note: Executing the following cell may take a few minutes

In [ ]:
# Create a subset of the data just for the 2016 presidential election
fgdb = r"C:\Users\albe9057\Documents\ArcGIS\Projects\Data Engineering and Visualization\Data Engineering and Visualization.gdb"
out_2016_fc_name = "county_elections_pres_2016"
out_2016_fc = data_2016_df.spatial.to_featureclass(os.path.join(fgdb, out_2016_fc_name))
out_2016_fc

# Step 3: Geoenrich election data
Goals:

Use Geoenrichment to bring demographic and socioeconomic variables to use in analysis
Geoenrichment in ArcGIS Pro allows us to add columns of data for each county that can help us analyze relationships and potentially model voter turnout. Geoenrichment can be performed using ArcPy, but we recommend that you use the GeoProcessing Enrich tool to explore potential variables that may help explain voter turnout. You may now reference the MOOC guidance for the steps covering how to geoenrich this data.

